In [2]:
import torch
from torch import nn
import numpy as np
import torch.nn.functional as F

In [4]:
with open('data/anna.txt','r') as f:
    text = f.read()          

In [6]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

In [13]:
chars = tuple(set(text))
int2chars = dict(enumerate(chars))
chars2int = { ch : ii for ii,ch in int2chars.items()}

encoded = np.array([chars2int[ch] for ch in text])

In [21]:
#data preprocessing: actually we will be doing the one hot encoding of the goven text values
def one_hot_encoder(arr, no_of_labels):
    one_hot = np.zeros((arr.size, no_of_labels), dtype= np.float32)
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] =1 
    one_hot =  one_hot.reshape(*arr.shape , no_of_labels)
    return one_hot

In [48]:
def get_batches(arr, batch_size, seq_length):
    
    batch_size_total = batch_size * seq_length
    
    # total number of batches that can be formed
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    
    arr = arr.reshape((batch_size, -1))
    # iterate through the array, one sequence at a time
    
    
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [49]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[22 82 23  4 57 38 29 68 18 62]
 [50 61 43 68 57 82 23 57 68 23]
 [38 43  1 68 61 29 68 23 68 33]
 [50 68 57 82 38 68  5 82 13 38]
 [68 50 23 34 68 82 38 29 68 57]
 [ 5 80 50 50 13 61 43 68 23 43]
 [68 28 43 43 23 68 82 23  1 68]
 [74 52 79 61 43 50 10 20 48 68]]

y
 [[82 23  4 57 38 29 68 18 62 62]
 [61 43 68 57 82 23 57 68 23 57]
 [43  1 68 61 29 68 23 68 33 61]
 [68 57 82 38 68  5 82 13 38 33]
 [50 23 34 68 82 38 29 68 57 38]
 [80 50 50 13 61 43 68 23 43  1]
 [28 43 43 23 68 82 23  1 68 50]
 [52 79 61 43 50 10 20 48 68 15]]


In [50]:
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

No GPU available, training on CPU; consider making n_epochs very small.


In [58]:
class charRNN(nn.Module):
    def __init__(self, tokens, n_hidden = 256, n_layers = 2,
                drop_prob = 0.5, lr = 0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        self.chars = tokens
        self.int2chars = dict(enumerate(chars))
        self.chars2int = {ch : ii for ch,ii in self.int2chars.items()}
        
        
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, dropout = drop_prob, batch_first = True)
        
        self.dropout = nn.Dropout(drop_prob)
        
        self.fc = nn.Linear(n_hidden, len(self.chars))
        
    def forward(self, x, hidden):
        r_out, hidden = self.lstm(x,hidden)
        out = self.dropout(r_out)
        out = out.contiguous().view(-1, self.n_hidden)
        out = self.fc(out)
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

In [59]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encoder(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encoder(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [61]:
n_hidden=512
n_layers=2

net = charRNN(chars, n_hidden, n_layers)
print(net)

charRNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


In [ ]:
batch_size = 128
seq_length = 100
n_epochs = 5 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/5... Step: 10... Loss: 3.2712... Val Loss: 3.2112
Epoch: 1/5... Step: 20... Loss: 3.1548... Val Loss: 3.1362
Epoch: 1/5... Step: 30... Loss: 3.1443... Val Loss: 3.1248
Epoch: 1/5... Step: 40... Loss: 3.1100... Val Loss: 3.1189
Epoch: 1/5... Step: 50... Loss: 3.1395... Val Loss: 3.1168
Epoch: 1/5... Step: 60... Loss: 3.1199... Val Loss: 3.1138
Epoch: 1/5... Step: 70... Loss: 3.1026... Val Loss: 3.1084
Epoch: 1/5... Step: 80... Loss: 3.1136... Val Loss: 3.0953
Epoch: 1/5... Step: 90... Loss: 3.0850... Val Loss: 3.0633
Epoch: 1/5... Step: 100... Loss: 3.0087... Val Loss: 2.9855
Epoch: 1/5... Step: 110... Loss: 2.9612... Val Loss: 2.9441
Epoch: 1/5... Step: 120... Loss: 2.8374... Val Loss: 2.8239
Epoch: 1/5... Step: 130... Loss: 2.7720... Val Loss: 2.7298
Epoch: 2/5... Step: 140... Loss: 2.6804... Val Loss: 2.6256
Epoch: 2/5... Step: 150... Loss: 2.6046... Val Loss: 2.5525
Epoch: 2/5... Step: 160... Loss: 2.5456... Val Loss: 2.4970
Epoch: 2/5... Step: 170... Loss: 2.4734... Val Lo

In [ ]:
model_name = 'rnn_20_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

In [ ]:
def predict(net, char, h=None, top_k=None):
    
        x = np.array([[net.char2int[char]]])
        x = one_hot_encoder(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [ ]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [ ]:
print(sample(net, 1000, prime='Anna', top_k=5))

In [ ]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_20_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

In [ ]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))